# Test my Neural Nets classes

Should do [Working efficiently with jupyter lab](https://florianwilhelm.info/2018/11/working_efficiently_with_jupyter_lab/)

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget
#%matplotlib inline

In [ ]:
# import Importing_Notebooks
import numpy as np
#from scipy import ndimage
import matplotlib.pyplot as plt
import dill

In [ ]:
from nn import Network, Layer, IdentityLayer, AffinityLayer, MapLayer
from nnbench import NNBench

___

## Tests

### One identity layer
See if the wheels turn:

In [ ]:
net = Network()
net.extend(IdentityLayer())
all(net(np.arange(3)) == np.arange(3))

It does not learn, as expected:

In [ ]:
facts = [(np.arange(2*n, 2*n+2), np.arange(2*n+1, 2*n-1, -1)) for n in range(3)]
net.learn(facts)

In [ ]:
net(np.arange(2,4))

### One map layer

In [ ]:
net = Network()
net.extend(MapLayer(lambda x: x+1, lambda d: 1))
all(net(np.arange(3)) == np.arange(3)+1)

It does not learn, as expected:

In [ ]:
net.learn(facts), all(net(np.arange(5)) == np.arange(5)+1), net(np.arange(2,4))

### One affine layer

In [ ]:
net = Network()
net.extend(AffinityLayer(2,2))

In [ ]:
t = net.layers[0]
t.M, t.b

#### Can it learn the identity transformation?

In [ ]:
bench = NNBench(net)
bench.checkpoint_net()
bench.learning_potential()

In [ ]:
bench.plot_learning(100)

In [ ]:
bench.ideal = lambda v: np.array([v[1], v[0]])
bench.knobs_plot_learning(100)

### Learn thru a map layer

This layer squares its input and divides by two:

In [ ]:
net = Network()
net.extend(AffinityLayer(2,2))

def dtanh(x):
    v = np.tanh(x)
    return (1+v)*(1-v)

net.extend(MapLayer(lambda x:x*x/2.0, lambda d:d))
#net.extend(MapLayer(np.tanh, dtanh))
bench = NNBench(net)
bench.checkpoint_net()

In [ ]:
net.layers[0].M, net.layers[0].b

#### Can it learn difference squared?

In [ ]:
bench.ideal = lambda v: [(v[0]-v[1])**2,0]
#bench.ideal = lambda v: [(v[0]>0)*2-1,(v[0]>v[1])*2-1]
bench.learning_potential()
#bench.knobs_plot_learning(100)

In [ ]:
bench.knobs_plot_learning(100)

### add a RELU

In [ ]:
bench.net.layers = []
bench.net.extend(AffinityLayer(2,2))
leak = 0
bench.net.extend(MapLayer(lambda x: (x*(1+leak/2)+abs(x)*(1-leak/2))/2, lambda d: [leak,1][1 if d>0 else 0]))
bench.net.layers

### XOR

In [ ]:
net = Network()
net.extend(AffinityLayer(2,2))

In [ ]:
t = net.layers[0]
t.M, t.b